# 멀티 에이전트 토론

멀티 에이전트 토론은 각 턴에서 에이전트들이 서로 응답을 교환하고, 
다른 에이전트들의 응답을 바탕으로 자신의 응답을 개선하는 멀티 턴 상호작용을 
시뮬레이션하는 멀티 에이전트 설계 패턴입니다.

이 예제는 [GSM8K 벤치마크](https://huggingface.co/datasets/openai/gsm8k)의 
수학 문제를 해결하기 위한 멀티 에이전트 토론 패턴의 구현을 보여줍니다.

이 패턴에는 두 가지 유형의 에이전트가 있습니다: 솔버 에이전트와 집계 에이전트입니다.
솔버 에이전트들은 [희소 통신 토폴로지를 통한 멀티 에이전트 토론 개선](https://arxiv.org/abs/2406.11776)에서 
설명된 기법에 따라 희소한 방식으로 연결됩니다.
솔버 에이전트는 수학 문제를 해결하고 서로 응답을 교환하는 역할을 합니다.
집계 에이전트는 수학 문제를 솔버 에이전트들에게 배포하고, 
최종 응답을 기다린 후, 응답들을 집계하여 최종 답을 얻는 역할을 합니다.

패턴의 작동 방식은 다음과 같습니다:
1. 사용자가 집계 에이전트에게 수학 문제를 보냅니다.
2. 집계 에이전트가 솔버 에이전트들에게 문제를 배포합니다.
3. 각 솔버 에이전트는 문제를 처리하고, 이웃들에게 응답을 발행합니다.
4. 각 솔버 에이전트는 이웃들의 응답을 사용하여 자신의 응답을 개선하고, 새로운 응답을 발행합니다.
5. 고정된 라운드 수만큼 4단계를 반복합니다. 마지막 라운드에서는 각 솔버 에이전트가 최종 응답을 발행합니다.
6. 집계 에이전트는 모든 솔버 에이전트들의 최종 응답을 다수결 투표로 집계하여 최종 답을 얻고, 그 답을 발행합니다.

우리는 브로드캐스트 API, 즉 {py:meth}`~autogen_core.BaseAgent.publish_message`를 사용하고,
토픽과 구독을 사용하여 통신 토폴로지를 구현할 것입니다.
작동 방식을 이해하려면 [토픽과 구독](../core-concepts/topic-and-subscription.md)을 읽어보세요.

In [1]:
import re
from dataclasses import dataclass
from typing import Dict, List

from autogen_core import (
    DefaultTopicId,
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TypeSubscription,
    default_subscription,
    message_handler,
)
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

## 메시지 프로토콜

먼저 에이전트들이 사용하는 메시지를 정의합니다.
`IntermediateSolverResponse`는 각 라운드에서 솔버 에이전트들 간에 교환되는 메시지이고,
`FinalSolverResponse`는 최종 라운드에서 솔버 에이전트들이 발행하는 메시지입니다.

In [2]:
@dataclass
class Question:
    content: str


@dataclass
class Answer:
    content: str


@dataclass
class SolverRequest:
    content: str
    question: str


@dataclass
class IntermediateSolverResponse:
    content: str
    question: str
    answer: str
    round: int


@dataclass
class FinalSolverResponse:
    answer: str

## 솔버 에이전트

솔버 에이전트는 수학 문제를 해결하고 다른 솔버 에이전트들과 응답을 교환하는 역할을 합니다.
`SolverRequest`를 받으면, 솔버 에이전트는 LLM을 사용하여 답을 생성합니다.
그런 다음 라운드 번호에 따라 `IntermediateSolverResponse` 또는 
`FinalSolverResponse`를 발행합니다.

솔버 에이전트에는 토픽 유형이 주어지며, 이는 에이전트가 
중간 응답을 발행해야 하는 토픽을 나타내는 데 사용됩니다. 이 토픽은 
이웃들이 이 에이전트의 응답을 받기 위해 구독합니다 -- 이것이 
어떻게 수행되는지는 나중에 보여드리겠습니다.

{py:meth}`~autogen_core.components.default_subscription`을 사용하여 
솔버 에이전트들이 기본 토픽을 구독하도록 합니다. 기본 토픽은 집계 에이전트가 
솔버 에이전트들로부터 최종 응답을 수집하는 데 사용됩니다.

In [3]:
@default_subscription
class MathSolver(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient, topic_type: str, num_neighbors: int, max_round: int) -> None:
        super().__init__("A debator.")
        self._topic_type = topic_type
        self._model_client = model_client
        self._num_neighbors = num_neighbors
        self._history: List[LLMMessage] = []
        self._buffer: Dict[int, List[IntermediateSolverResponse]] = {}
        self._system_messages = [
            SystemMessage(
                content=(
                    "You are a helpful assistant with expertise in mathematics and reasoning. "
                    "Your task is to assist in solving a math reasoning problem by providing "
                    "a clear and detailed solution. Limit your output within 100 words, "
                    "and your final answer should be a single numerical number, "
                    "in the form of {{answer}}, at the end of your response. "
                    "For example, 'The answer is {{42}}.'"
                )
            )
        ]
        self._round = 0
        self._max_round = max_round

    @message_handler
    async def handle_request(self, message: SolverRequest, ctx: MessageContext) -> None:
        # Add the question to the memory.
        self._history.append(UserMessage(content=message.content, source="user"))
        # Make an inference using the model.
        model_result = await self._model_client.create(self._system_messages + self._history)
        assert isinstance(model_result.content, str)
        # Add the response to the memory.
        self._history.append(AssistantMessage(content=model_result.content, source=self.metadata["type"]))
        print(f"{'-'*80}\nSolver {self.id} round {self._round}:\n{model_result.content}")
        # Extract the answer from the response.
        match = re.search(r"\{\{(\-?\d+(\.\d+)?)\}\}", model_result.content)
        if match is None:
            raise ValueError("The model response does not contain the answer.")
        answer = match.group(1)
        # Increment the counter.
        self._round += 1
        if self._round == self._max_round:
            # If the counter reaches the maximum round, publishes a final response.
            await self.publish_message(FinalSolverResponse(answer=answer), topic_id=DefaultTopicId())
        else:
            # Publish intermediate response to the topic associated with this solver.
            await self.publish_message(
                IntermediateSolverResponse(
                    content=model_result.content,
                    question=message.question,
                    answer=answer,
                    round=self._round,
                ),
                topic_id=DefaultTopicId(type=self._topic_type),
            )

    @message_handler
    async def handle_response(self, message: IntermediateSolverResponse, ctx: MessageContext) -> None:
        # Add neighbor's response to the buffer.
        self._buffer.setdefault(message.round, []).append(message)
        # Check if all neighbors have responded.
        if len(self._buffer[message.round]) == self._num_neighbors:
            print(
                f"{'-'*80}\nSolver {self.id} round {message.round}:\nReceived all responses from {self._num_neighbors} neighbors."
            )
            # Prepare the prompt for the next question.
            prompt = "These are the solutions to the problem from other agents:\n"
            for resp in self._buffer[message.round]:
                prompt += f"One agent solution: {resp.content}\n"
            prompt += (
                "Using the solutions from other agents as additional information, "
                "can you provide your answer to the math problem? "
                f"The original math problem is {message.question}. "
                "Your final answer should be a single numerical number, "
                "in the form of {{answer}}, at the end of your response."
            )
            # Send the question to the agent itself to solve.
            await self.send_message(SolverRequest(content=prompt, question=message.question), self.id)
            # Clear the buffer.
            self._buffer.pop(message.round)

## 집계 에이전트

집계 에이전트는 사용자 질문을 처리하고 
수학 문제를 솔버 에이전트들에게 배포하는 역할을 합니다.

집계 에이전트는 {py:meth}`~autogen_core.components.default_subscription`을 사용하여 
기본 토픽을 구독합니다. 기본 토픽은 사용자 질문을 받고, 
솔버 에이전트들로부터 최종 응답을 받으며, 
최종 답을 사용자에게 다시 발행하는 데 사용됩니다.

멀티 에이전트 토론을 하위 구성요소로 격리하려는 더 복잡한 애플리케이션에서는 
{py:meth}`~autogen_core.components.type_subscription`을 사용하여 
집계자-솔버 통신을 위한 특정 토픽 유형을 설정하고,
솔버와 집계자 모두 해당 토픽 유형에 발행하고 구독하도록 해야 합니다.

In [4]:
@default_subscription
class MathAggregator(RoutedAgent):
    def __init__(self, num_solvers: int) -> None:
        super().__init__("Math Aggregator")
        self._num_solvers = num_solvers
        self._buffer: List[FinalSolverResponse] = []

    @message_handler
    async def handle_question(self, message: Question, ctx: MessageContext) -> None:
        print(f"{'-'*80}\nAggregator {self.id} received question:\n{message.content}")
        prompt = (
            f"Can you solve the following math problem?\n{message.content}\n"
            "Explain your reasoning. Your final answer should be a single numerical number, "
            "in the form of {{answer}}, at the end of your response."
        )
        print(f"{'-'*80}\nAggregator {self.id} publishes initial solver request.")
        await self.publish_message(SolverRequest(content=prompt, question=message.content), topic_id=DefaultTopicId())

    @message_handler
    async def handle_final_solver_response(self, message: FinalSolverResponse, ctx: MessageContext) -> None:
        self._buffer.append(message)
        if len(self._buffer) == self._num_solvers:
            print(f"{'-'*80}\nAggregator {self.id} received all final answers from {self._num_solvers} solvers.")
            # Find the majority answer.
            answers = [resp.answer for resp in self._buffer]
            majority_answer = max(set(answers), key=answers.count)
            # Publish the aggregated response.
            await self.publish_message(Answer(content=majority_answer), topic_id=DefaultTopicId())
            # Clear the responses.
            self._buffer.clear()
            print(f"{'-'*80}\nAggregator {self.id} publishes final answer:\n{majority_answer}")

## 토론 설정

이제 4개의 솔버 에이전트와 1개의 집계 에이전트로 멀티 에이전트 토론을 설정하겠습니다.
솔버 에이전트들은 아래 그림과 같이 희소한 방식으로 연결됩니다:

```
A --- B
|     |
|     |
D --- C
```

각 솔버 에이전트는 다른 두 솔버 에이전트와 연결됩니다. 
예를 들어, 에이전트 A는 에이전트 B와 C에 연결됩니다.

먼저 런타임을 생성하고 에이전트 유형을 등록해보겠습니다.

In [5]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)

AZURE_DEPLOYMENT = "gpt-4o-mini"
MODEL = "gpt-4o-mini"
API_VERSION = "2024-06-01"

clientA = AzureOpenAIChatCompletionClient(
    azure_deployment=AZURE_DEPLOYMENT,
    model=MODEL,
    api_version=API_VERSION,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

clientB = AzureOpenAIChatCompletionClient(
    azure_deployment=AZURE_DEPLOYMENT,
    model=MODEL,
    api_version=API_VERSION,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

clientC = AzureOpenAIChatCompletionClient(
    azure_deployment=AZURE_DEPLOYMENT,
    model=MODEL,
    api_version=API_VERSION,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

clientD = AzureOpenAIChatCompletionClient(
    azure_deployment=AZURE_DEPLOYMENT,
    model=MODEL,
    api_version=API_VERSION,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [6]:
runtime = SingleThreadedAgentRuntime()
await MathSolver.register(
    runtime,
    "MathSolverA",
    lambda: MathSolver(
        model_client=clientA,
        topic_type="MathSolverA",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathSolver.register(
    runtime,
    "MathSolverB",
    lambda: MathSolver(
        model_client=clientB,
        topic_type="MathSolverB",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathSolver.register(
    runtime,
    "MathSolverC",
    lambda: MathSolver(
        model_client=clientC,
        topic_type="MathSolverC",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathSolver.register(
    runtime,
    "MathSolverD",
    lambda: MathSolver(
        model_client=clientD,
        topic_type="MathSolverD",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathAggregator.register(runtime, "MathAggregator", lambda: MathAggregator(num_solvers=4))

AgentType(type='MathAggregator')

이제 {py:class}`~autogen_core.components.TypeSubscription`을 사용하여 솔버 에이전트 토폴로지를 생성하겠습니다.
이는 각 솔버 에이전트의 발행 토픽 유형을 이웃 에이전트 유형에 매핑합니다.

In [7]:
# Subscriptions for topic published to by MathSolverA.
await runtime.add_subscription(TypeSubscription("MathSolverA", "MathSolverD"))
await runtime.add_subscription(TypeSubscription("MathSolverA", "MathSolverB"))

# Subscriptions for topic published to by MathSolverB.
await runtime.add_subscription(TypeSubscription("MathSolverB", "MathSolverA"))
await runtime.add_subscription(TypeSubscription("MathSolverB", "MathSolverC"))

# Subscriptions for topic published to by MathSolverC.
await runtime.add_subscription(TypeSubscription("MathSolverC", "MathSolverB"))
await runtime.add_subscription(TypeSubscription("MathSolverC", "MathSolverD"))

# Subscriptions for topic published to by MathSolverD.
await runtime.add_subscription(TypeSubscription("MathSolverD", "MathSolverC"))
await runtime.add_subscription(TypeSubscription("MathSolverD", "MathSolverA"))

# All solvers and the aggregator subscribe to the default topic.

## 수학 문제 해결

이제 토론을 실행하여 수학 문제를 해결해보겠습니다.
기본 토픽에 `SolverRequest`를 발행하면, 
집계 에이전트가 토론을 시작합니다.

In [8]:
question = "나탈리아는 4월에 48명의 친구에게 클립을 판매했고, 5월에는 그 절반의 클립을 판매했습니다. 나탈리아는 4월과 5월에 총 몇 개의 클립을 판매했나요?"
runtime.start()
await runtime.publish_message(Question(content=question), DefaultTopicId())
await runtime.stop_when_idle()

--------------------------------------------------------------------------------
Aggregator MathAggregator/default received question:
나탈리아는 4월에 48명의 친구에게 클립을 판매했고, 5월에는 그 절반의 클립을 판매했습니다. 나탈리아는 4월과 5월에 총 몇 개의 클립을 판매했나요?
--------------------------------------------------------------------------------
Aggregator MathAggregator/default publishes initial solver request.
--------------------------------------------------------------------------------
Solver MathSolverA/default round 0:
나탈리아는 4월에 48개의 클립을 판매했습니다. 5월에 판매한 클립의 수는 4월의 절반이므로, 48의 절반은 24입니다. 

따라서 4월과 5월에 판매한 클립의 총 수는 48 + 24 = 72입니다. 

따라서 답은 {{72}}입니다.
--------------------------------------------------------------------------------
Solver MathSolverD/default round 0:
나탈리아가 4월에 판매한 클립 수는 48개입니다. 5월에는 4월 판매량의 절반인 24개를 판매했습니다. 

총 판매량은 4월과 5월의 판매량을 더한 것입니다:
48 + 24 = 72

따라서, 나탈리아는 4월과 5월에 총 72개의 클립을 판매했습니다. 

The answer is {{72}}.
--------------------------------------------------------------------------------
Solver MathSolverA/d